In [194]:
import pdfplumber
import pandas as pd

# Abre o PDF e extrai as tabelas
with pdfplumber.open("cardapio_junho.pdf") as pdf:
    page = pdf.pages[0]
    tables = page.extract_tables()
    
    # Processar o cabeçalho (tabela 0)
    # Sabemos que as posições com o texto dos dias são:
    # coluna 1, 4, 7, 10 e 13 (evitar colunas vazias)
    # e as respectivas partes na segunda linha.
    header0 = tables[0]
    day_indices = [1, 4, 7, 10, 13]
    day_names = []
    for idx in day_indices:
        # Some espaço se houver None
        part1 = header0[0][idx] or ""
        part2 = header0[1][idx] or ""
        combined = f"{part1} {part2}".strip()
        day_names.append(combined)
    
    # adiciona o identificador categoria pra primeira coluna    
    colunas_finais = ['Categoria'] + day_names
    
    
    # Processar o corpo da tabela (tabela 1)
    body_raw = tables[1]
    # Remover linhas totalmente vazias
    clean_body = [row for row in body_raw if row and any(cell and cell.strip() for cell in row)]
    
    # Devido à inconsistência, vamos ajustar cada linha.
    # Nossa estratégia:
    # - Utilizar apenas as 6 primeiras posições: a coluna 0 é a Categoria e as próximas 5 devem representar os dias.
    # - Se a célula referente à sexta-feira (índice 5) estiver vazia,
    #   mas houver conteúdo na coluna seguinte (índice 6), usamos essa informação.
    def adjust_row(row):
        # Se a linha tiver menos de 7 colunas, vamos preenchê-la com None
        row = row + [None] * (7 - len(row))
        # Se a posição 5 estiver vazia e a 6 tiver dado, substituímos:
        if (not row[5] or row[5].strip() == "") and row[6] and row[6].strip() != "":
            row[5] = row[6]
        # Retornamos apenas as primeiras 6 colunas: [Categoria, Segunda, Terça, Quarta, Quinta, Sexta]
        return row[:6]
    
    adjusted_body = [adjust_row(row) for row in clean_body if len(row) >= 6]
    
    # Criação do DataFrame com as colunas finais
    df = pd.DataFrame(adjusted_body, columns=colunas_finais)
    
    # Preenche as linhas onde a coluna Categoria está vazia com o valor anterior
    df['Categoria'] = df['Categoria'].ffill()




In [195]:
df_agrupado = (
    df.groupby('Categoria', sort=False)
        .agg(lambda col: ' e '.join(dict.fromkeys(filter(None, col))))
        .reset_index()
)


In [196]:
# Exibe resultado
df_agrupado

,Categoria,"segunda-feira, 2 de junho de 2025","terça-feira, 3 de junho de 2025","quarta-feira, 4 de junho de 2025","quinta-feira, 5 de junho de 2025","sexta-feira, 6 de junho de 2025"
0,Saladas,Acelga com tomate picado e Beterraba cozida,Chicória com cenoura\nralada e Chuchu colorido...,Alface com rabanete e Grão de bico com cenoura...,Almeirão com cenoura e Repolho cozido\nFruta,Alface com repolho e Abobrinha colorida
1,Acompanhamento,Arroz Branco e Feijão Preto,Arroz Branco e Feijão Carioca,Arroz Branco/arroz\ncolorido e Feijão Preto,Arroz Branco e Feijão Preto,Arroz Branco e Feijão Preto
2,Guarnição,Aipim souté,Espaguete ao molho sugo,Quirerinha temperada,Batata doce\ncaramelada,Macarrão com ervas\nfinas
3,Vegetariano,Omelete de forno,Entreveiro de legumes,Escondidinho de cabotiá\ncom proteína de soja,Mini pizza de\nabobrinha,Ovo ao molho
4,Prato Principal,Iscas suínas refogadas,Espetinho de frango,Bife a tomatelli,Pernil grelhado,Frango assado


In [197]:
print(df_agrupado)

         Categoria            segunda-feira, 2 de junho de 2025  \
0          Saladas  Acelga com tomate picado e Beterraba cozida   
1   Acompanhamento                  Arroz Branco e Feijão Preto   
2        Guarnição                                  Aipim souté   
3      Vegetariano                             Omelete de forno   
4  Prato Principal                       Iscas suínas refogadas   

                     terça-feira, 3 de junho de 2025  \
0  Chicória com cenoura\nralada e Chuchu colorido...   
1                      Arroz Branco e Feijão Carioca   
2                            Espaguete ao molho sugo   
3                              Entreveiro de legumes   
4                                Espetinho de frango   

                    quarta-feira, 4 de junho de 2025  \
0  Alface com rabanete e Grão de bico com cenoura...   
1        Arroz Branco/arroz\ncolorido e Feijão Preto   
2                               Quirerinha temperada   
3      Escondidinho de cabotiá\ncom 

In [198]:
with pdfplumber.open("cardapio_junho.pdf") as pdf:
    for i, page in enumerate(pdf.pages, start=1):
        if i == 3:
            continue
        print(f"\n📄 Página {i}")
        tables = page.extract_tables()
        if not tables:
            print("⚠️ Nenhuma tabela encontrada nesta página.")
            continue
        for t_idx, table in enumerate(tables):
            print(f"\nTabela {t_idx} da página {i}")
            for row in table:
                print(row)



📄 Página 1

Tabela 0 da página 1
['', 'segunda-feira, 2 de junho de', '', '', 'terça-feira, 3 de junho de', '', '', 'quarta-feira, 4 de junho', '', '', 'quinta-feira, 5 de', '', '', 'sexta-feira, 6 de junho', '']
[None, '2025', None, None, '2025', None, None, 'de 2025', None, None, 'junho de 2025', None, None, 'de 2025', None]

Tabela 1 da página 1
['Saladas', 'Acelga com tomate picado', 'Chicória com cenoura\nralada', 'Alface com rabanete', 'Almeirão com cenoura', '', 'Alface com repolho', '']
[None, None, None, None, None, None, '', None]
[None, 'Beterraba cozida', 'Chuchu colorido\nFruta', 'Grão de bico com cenoura\ncozida', 'Repolho cozido\nFruta', 'Abobrinha colorida', None, None]
[None, None, None, None, None, None, 'Abobrinha colorida', None]
[None, None, None, None, None, None, '', None]
[None, None, None, None, None, '', '', '']
['Acompanhamento', 'Arroz Branco', 'Arroz Branco', 'Arroz Branco/arroz\ncolorido', 'Arroz Branco', 'Arroz Branco', None, None]
[None, 'Feijão Preto',